In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False

앙상블(Ensemble)

지도 학습은 피쳐 데이터와 타겟 데이터를 이용해서 전체 데이터를 분류하는 학습 방법이다.  
지도 학습에 사용되는 데이터는 각 피쳐 데이터마다 타겟 변수가 올바르게 할당된 라벨링 데이터였고 이를 기반으로 학습 알고리즘을 생성하는 과정을 거쳤었다.

앙상블 학습의 핵심 아이디어는 트레이닝 데이터를 기반으로 분류 모델을 여러 개 만들고 서로 비교하는 것이다.  
앙상블 학습 과정에서 만든 개별 분류 모형을 분류기(classifier)라고 하고 여러 개의 분류기를 결합함으로써 개별적인 분류기보다 성능이 뛰어난 최종 분류기를 만드는 것이 앙상블 학습의 목적이다.

보팅(Voting)  
여러 개의 분류 모형의 결과를 대상으로 투표를 통해서 최종 클래스의 레이블을 정하는 방법이다.

분류기가 10개 있다고 했을 때, 특정 데이터에 대해서 7개의 분류기는 클래스1 이라고 예측하고, 나머지 3개의 분류기는 클래스2 라고 예측했을 때 클래스1이 가장 높은 득표수를 보이므로 최종적으로 클래스1로 예측하는 것이다. 이를 다수결 투표라고 하는데 이와 비슷한 방법으로 다수결이 아닌 절반 이상의 분류기의 표를 얻어야 하는 과반수 투표 방식이 있다.

개별 분류기는 지도 학습 방법 중 k최근접 이웃, 로지스틱 회귀, 나이브베이즈, 의사 결정 트리, 서포트 벡터 머신 등 여러 가지 알고리즘을 사용해서 다양한 분류 모델을 만들어 사용할 수 있다.

보팅 알고리즘을 활용해 붓꽃 데이터를 분류하는 앙상블 모형을 만든다.

In [2]:
# 데이터 불러오기
from sklearn import datasets # 사이킷런이 제공하는 데이터셋을 사용하기 위해 import 한다.
raw_iris = datasets.load_iris() # 붓꽃 데이터를 불러온다.

In [3]:
# 피쳐, 타겟 데이터 저장
X = raw_iris.data # 피쳐 데이터를 저장한다.
y = raw_iris.target # 타겟 데이터를 저장한다.
X.shape

(150, 4)

In [4]:
# 트레이닝, 테스트 데이터 분할
from sklearn.model_selection import train_test_split # 트레이닝, 테스트 데이터 분할을 위해 import 한다.
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)
print(X_tn.shape, y_tn.shape, X_te.shape, y_te.shape) # 트레이닝 데이터와 테스트 데이터로 분할한다.

(112, 4) (112,) (38, 4) (38,)


In [5]:
# 데이터 표준화
from sklearn.preprocessing import StandardScaler # 데이터 표준화를 위해 import 한다.
std_scale = StandardScaler() # 표준화 스케일러 객체를 만든다.
# 표준화는 트레이닝 데이터를 기반으로 실행하므로 트레이닝 피쳐 데이터 X_tn을 표준화 스케일러에 적합시킨다.
X_tn_std = std_scale.fit_transform(X_tn) # 트레이닝 데이터를 적합 후 표준화 한다.
X_te_std = std_scale.transform(X_te) # 테스트 데이터를 표준화 한다.

In [22]:
# 모델 생성 후 데이터 학습
from sklearn.linear_model import LogisticRegression # 로지스틱 회귀 알고리즘을 사용하기 위해 import 한다.
from sklearn.svm import SVC # 서포트 벡터 머신 알고리즘을 사용하기 위해 import 한다.
from sklearn.naive_bayes import GaussianNB # 가우시안 나이브베이즈 알고리즘을 사용하기 위해 import 한다.
from sklearn.ensemble import VotingClassifier # 보팅 알고리즘을 사용하기 위해 import 한다.

clf1 = LogisticRegression() # 로지스틱 회귀 알고리즘을 사용하는 개별 분류기를 만든다.
clf2 = SVC(kernel='linear') # 서포트 벡터 머신 알고리즘을 사용하는 개별 분류기를 만든다.
clf3 = GaussianNB() # 가우시안 나이브베이즈 알고리즘을 사용하는 개별 분류기를 만든다.

# 위 3개의 개별 분류기를 이용해서 보팅 모델을 만든다.
# estimators 옵션으로 보팅 모델에서 사용할 개별 분류기를 지정한다.
# voting 옵션으로 투표 방식을 지정한다. 투표 방식은 hard(과반수 투표, 기본값)와 soft(다수결 투표)가 있다.
# weights 옵션으로 개별 분류기에 가중치를 지정해서 가중치 투표를 할 수 있다.
clf_voting = VotingClassifier(estimators=[('lr', clf1), ('svm', clf2), ('gnb', clf3)], voting='hard', 
                              weights=[1, 1, 1]) # 보팅 모델을 만든다.
# 표준화된 트레이닝 피쳐 데이터 X_tn_std와 트레이닝 타겟 데이터 y_tn를 모델에 넣어서 보팅 알고리즘을 학습시킨다.
clf_voting.fit(X_tn_std, y_tn)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('svm', SVC(kernel='linear')),
                             ('gnb', GaussianNB())],
                 weights=[1, 1, 1])

In [23]:
# 테스트 데이터 예측
# 표준화된 테스트 데이터 X_te_std로 예측한다.
pred_voting = clf_voting.predict(X_te_std)
print(pred_voting)

[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 2]


In [24]:
# 정확도 평가
from sklearn.metrics import accuracy_score # 모델의 정확도를 평가하기 위해 import 한다.
# accuracy_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_voting)를 넘겨 정확도를 계산한다.
accuracy = accuracy_score(y_te, pred_voting)
print(accuracy)

0.9736842105263158


In [25]:
# 정밀도 평가
from sklearn.metrics import precision_score # 모델의 정밀도를 평가하기 위해 import 한다.
# precision_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_voting)를 넘겨 정밀도를 계산한다.
precision = precision_score(y_te, pred_voting, average=None)
print(precision)

[1.  1.  0.9]


In [26]:
# 재현율 평가
from sklearn.metrics import recall_score # 모델의 재현율를 평가하기 위해 import 한다.
# recall_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_voting)를 넘겨 재현율를 계산한다.
recall = recall_score(y_te, pred_voting, average=None)
print(recall)

[1.     0.9375 1.    ]


In [27]:
# f1 score 평가
from sklearn.metrics import f1_score # 모델의 f1 score를 평가하기 위해 import 한다.
# f1_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_voting)를 넘겨 f1 score를 계산한다.
f1 = f1_score(y_te, pred_voting, average=None)
print(f1)

[1.         0.96774194 0.94736842]


In [28]:
# 혼돈 행렬 확인
from sklearn.metrics import confusion_matrix # 혼돈 행렬을 만들기 위해 import 한다.
# confusion_matrix() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_voting)를 넘겨 혼돈 행렬을 만든다.
conf_matrix = confusion_matrix(y_te, pred_voting)
print(conf_matrix)

[[13  0  0]
 [ 0 15  1]
 [ 0  0  9]]


In [30]:
# 분류 리포트 확인
from sklearn.metrics import classification_report # 분류 리포트를 출력하기 위해 import 한다.
# classification_report() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_voting)를 넘겨 분류 리포트를 만든다.
target_names = raw_iris.target_names
class_report = classification_report(y_te, pred_voting, target_names=target_names)
print(class_report)

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        13
  versicolor       1.00      0.94      0.97        16
   virginica       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38

